In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
import numpy as np
import joblib

# Define paths and parameters
train_dir = 'training_data'
test_dir = 'testing_Data'
image_height = 32  
image_width = 32  
batch_size = 32

# Create data generators for loading images
datagen = ImageDataGenerator(rescale=1./255)

train_dataset = datagen.flow_from_directory(
    train_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

test_dataset = datagen.flow_from_directory(
    test_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

# Load the pre-trained VGG16 model and use it for feature extraction
base_model = tf.keras.applications.VGG16(include_top=False, input_shape=(image_height, image_width, 3))
base_model.trainable = False

# Function to extract features using the pre-trained model
def extract_features(dataset):
    features = base_model.predict(dataset)
    features = features.reshape((features.shape[0], -1))  # Flatten the features
    return features

# Extract features for training and testing datasets
train_features = extract_features(train_dataset)
train_labels = train_dataset.classes
test_features = extract_features(test_dataset)
test_labels = test_dataset.classes

# Train the SVM with a linear kernel
svm = SVC(kernel='linear')
svm.fit(train_features, train_labels)

# Save the SVM model
joblib.dump(svm, 'svm_model.joblib')

# Predict the labels for the test dataset
y_pred = svm.predict(test_features)

# Calculate metrics
accuracy = accuracy_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred, average='weighted')
recall = recall_score(test_labels, y_pred, average='weighted')
f1 = f1_score(test_labels, y_pred, average='weighted')
conf_matrix = confusion_matrix(test_labels, y_pred)

# Print the metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')
print(conf_matrix)


Found 20628 images belonging to 36 classes.
Found 1008 images belonging to 36 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


c:\Users\Ashad InnO\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


645/645 ━━━━━━━━━━━━━━━━━━━━ 152s 236ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 201ms/step
Accuracy: 0.9890873015873016
Precision: 0.9903439153439153
Recall: 0.9890873015873016
F1 Score: 0.9889455782312926
Confusion Matrix:
[[27  0  0 ...  0  0  0]
 [ 0 28  0 ...  0  0  0]
 [ 0  0 28 ...  0  0  0]
 ...
 [ 0  0  0 ... 28  0  0]
 [ 0  0  0 ...  0 28  0]
 [ 0  0  0 ...  0  0 28]]
